In [1]:
const X = 1
const Y = 2
const Z = 3
const DX = 4
const DY = 5
const DZ = 6

6

In [2]:
function parse_line(l)
    (s,e) = split(l, "~")
    p = split(s, ',')
    x = parse(Int, p[1])
    y = parse(Int, p[2])
    z = parse(Int, p[3])
    deltas = split(e, ',')
    dx = parse(Int, deltas[1])-x
    dy = parse(Int, deltas[2])-y
    dz = parse(Int, deltas[3])-z 
    
    #if (dx < 0) || (dy < 0) || (dz < 0)
    #    println("Foudna a negative delta for line $l")
    #end
    #if ((dx > 0) && (dy > 0)) || ((dy < 0) && (dz > 0)) || ((dz < 0) && (dx > 0))
    #    println("Found a 2D block for line $l")
    #end
    return (x,y,z,dx,dy,dz)
    # return (x,y,z,dx > 0 ? dx : (dy > 0 ? dy : dz),dx > 0 ? X : (dy > 0 ? Y : Z))

end

parse_line (generic function with 1 method)

In [3]:
parse_line("0,2,3~2,2,3")

(0, 2, 3, 2, 0, 0)

In [4]:
using DataStructures

In [6]:
(nb_blocks) = open("22_1.txt") do f
    nb = 0

    blocks = DefaultDict(() -> Dict())

    all_lines = [parse_line(l) for l in eachline(f)]

    for (idx,l) in enumerate(sort!(all_lines, by = x -> x[3]))
        blocks[idx]["pos"] = l
        blocks[idx]["blocking"] = []
        blocks[idx]["blocked_by"] = []
    end

    # Keeping track of which cell is occupied
    position_to_block = Dict()
    blah = 1
    for b in keys(blocks)
        println("Block $blah / $(length(all_lines)) ongoing ($b => $(blocks[b]))")
        x = blocks[b]["pos"][X]
        y = blocks[b]["pos"][Y]
        z = blocks[b]["pos"][Z]
        for dx in range(0, blocks[b]["pos"][DX])
            for dy in range(0, blocks[b]["pos"][DY])
                for dz in range(0, blocks[b]["pos"][DZ])
                    position_to_block[(x+dx, y+dy, z+dz)] = b
                end
            end
        end
        blah += 1
    end

    println("All checks done")

    for a_block in range(1, length(blocks))
        # println(a_block)
        deltaz = 0
        blocking = []
        x = blocks[a_block]["pos"][X]
        y = blocks[a_block]["pos"][Y]
        z = blocks[a_block]["pos"][Z]
        not_blocked = (z != 1)
        while not_blocked
            for dx in range(0, blocks[a_block]["pos"][DX])
                for dy in range(0, blocks[a_block]["pos"][DY])
                    if z-deltaz-1 == 0.
                        println("$(a_block): $(blocks[a_block]) blocked by ground")
                        not_blocked = false
                    elseif (x+dx, y+dy, z-deltaz-1) in keys(position_to_block)
                        push!(blocking, position_to_block[(x+dx, y+dy, z-deltaz-1)])
                        println("$(a_block) blocked by $(position_to_block[(x+dx, y+dy, z-deltaz-1)])")
                        not_blocked = false
                    end
                end
            end
            if not_blocked
                deltaz += 1
            end
        end

        for dx in range(0, blocks[a_block]["pos"][DX])
            for dy in range(0, blocks[a_block]["pos"][DY])
                for dz in range(0, blocks[a_block]["pos"][DZ])
                    delete!(position_to_block,(x+dx, y+dy, z+dz))
                    position_to_block[(x+dx, y+dy, z+dz-deltaz)] = a_block
                end
            end
        end

        for i in blocking
            push!(blocks[a_block]["blocked_by"], i)
            push!(blocks[i]["blocking"], a_block)
        end

    end

    useless = Set()
    important = Set()

    for i in keys(blocks)
        for b in blocks[i]["blocked_by"]
            push!(useless, b)
        end
        if length(blocks[i]["blocked_by"]) == 1
            push!(important, blocks[i]["blocked_by"][1])
        end
        if length(blocks[i]["blocking"]) == 0
            push!(useless, i)
        end
    end

    (length(setdiff(useless, important)))
end

Block 1 / 1377 ongoing (1144 => Dict{Any, Any}("blocked_by" => Any[], "pos" => (7, 0, 247, 0, 1, 0), "blocking" => Any[]))
Block 2 / 1377 ongoing (1175 => Dict{Any, Any}("blocked_by" => Any[], "pos" => (2, 6, 253, 3, 0, 0), "blocking" => Any[]))
Block 3 / 1377 ongoing (719 => Dict{Any, Any}("blocked_by" => Any[], "pos" => (1, 8, 156, 1, 0, 0), "blocking" => Any[]))
Block 4 / 1377 ongoing (1028 => Dict{Any, Any}("blocked_by" => Any[], "pos" => (6, 7, 223, 2, 0, 0), "blocking" => Any[]))
Block 5 / 1377 ongoing (699 => Dict{Any, Any}("blocked_by" => Any[], "pos" => (6, 1, 152, 2, 0, 0), "blocking" => Any[]))
Block 6 / 1377 ongoing (831 => Dict{Any, Any}("blocked_by" => Any[], "pos" => (8, 7, 178, 0, 2, 0), "blocking" => Any[]))
Block 7 / 1377 ongoing (1299 => Dict{Any, Any}("blocked_by" => Any[], "pos" => (4, 1, 280, 0, 2, 0), "blocking" => Any[]))
Block 8 / 1377 ongoing (1074 => Dict{Any, Any}("blocked_by" => Any[], "pos" => (1, 0, 231, 2, 0, 0), "blocking" => Any[]))
Block 9 / 1377 ongo

645

In [53]:
nb_blocks

Set{Any} with 5 elements:
  5
  4
  7
  2
  3

In [25]:
# test_blocks = nb_blocks

DefaultDict{Any, Any, var"#54#58"} with 8 entries:
  5 => Dict{Any, Any}("blocked_by"=>Any[], "pos"=>(2, 0, 5, 0, 2, 0), "blocking…
  4 => Dict{Any, Any}("blocked_by"=>Any[], "pos"=>(0, 0, 4, 0, 2, 0), "blocking…
  6 => Dict{Any, Any}("blocked_by"=>Any[], "pos"=>(0, 1, 6, 2, 0, 0), "blocking…
  7 => Dict{Any, Any}("blocked_by"=>Any[], "pos"=>(1, 1, 8, 0, 0, 1), "blocking…
  2 => Dict{Any, Any}("blocked_by"=>Any[], "pos"=>(0, 0, 2, 2, 0, 0), "blocking…
  0 => Dict{Any, Any}("blocked_by"=>Any[], "pos"=>[0, 0, 0, 10000, 10000, 0], "…
  3 => Dict{Any, Any}("blocked_by"=>Any[], "pos"=>(0, 2, 3, 2, 0, 0), "blocking…
  1 => Dict{Any, Any}("blocked_by"=>Any[], "pos"=>(1, 0, 1, 0, 2, 0), "blocking…

In [40]:
println("791: youre answer is too high")

791: youre answer is too high


In [47]:
println("645: youre answer is too high")

645: youre answer is too high


In [45]:
Set(0)

Set{Int64} with 1 element:
  0